In [15]:
#!!!!注意，請先執行這個區塊，依網頁指示通過授權後載入 Google 雲端硬碟, 再執行底下的程式
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 13.2 LSTM (長短期記憶神經網路)

#### 載入套件

In [16]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, SpatialDropout1D
from tensorflow.keras.layers import LSTM # new! 
from tensorflow.keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import roc_auc_score 
import matplotlib.pyplot as plt 
%matplotlib inline

#### LSTM 分類模型的超參數

In [17]:
# 輸出目錄名稱：
output_dir = '/content/drive/MyDrive/Colab Notebooks/F1383_Sample/Ch13/model_output/LSTM' #註：請記得依你存放的位置彈性修改路徑

# 訓練
epochs = 4
batch_size = 128

# 詞向量空間
n_dim = 64 
n_unique_words = 10000 
max_review_length = 100 
pad_type = trunc_type = 'pre'
drop_embed = 0.2 

# LSTM 層架構
n_lstm = 256 
drop_lstm = 0.2

# 密集層架構: 
# n_dense = 256
# dropout = 0.2

#### 載入資料集

In [18]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=n_unique_words)

#### 預處理

In [19]:
x_train = pad_sequences(x_train, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)
x_test = pad_sequences(x_test, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)

#### 13.2.2 LSTM 分類模型架構

In [20]:
model = Sequential()
model.add(Embedding(n_unique_words, n_dim, input_length=max_review_length)) 
model.add(SpatialDropout1D(drop_embed))
model.add(LSTM(n_lstm, dropout=drop_lstm))
# model.add(Dense(n_dense, activation='relu')) 
# model.add(Dropout(dropout))
model.add(Dense(1, activation='sigmoid'))

In [21]:
model.summary() 

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 64)           640000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 100, 64)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               328704    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 968,961
Trainable params: 968,961
Non-trainable params: 0
_________________________________________________________________


#### 編譯模型

In [22]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [23]:
modelcheckpoint = ModelCheckpoint(filepath=output_dir+"/weights.{epoch:02d}.hdf5")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

#### 訓練模型

In [24]:
#註：由於神經網路的初始權重參數是隨機設定的, 參雜了隨機性, 因此底下 (或您重跑一次) 的結果不會與書中完全一樣, 但模型的能力是相近的
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test), callbacks=[modelcheckpoint])

Epoch 1/4
196/196 [==============================] - 210s 1s/step - loss: 0.5136 - accuracy: 0.7305 - val_loss: 0.3645 - val_accuracy: 0.8388
Epoch 2/4
196/196 [==============================] - 207s 1s/step - loss: 0.2956 - accuracy: 0.8786 - val_loss: 0.3746 - val_accuracy: 0.8428
Epoch 3/4
196/196 [==============================] - 208s 1s/step - loss: 0.2366 - accuracy: 0.9051 - val_loss: 0.3685 - val_accuracy: 0.8425
Epoch 4/4
196/196 [==============================] - 208s 1s/step - loss: 0.1940 - accuracy: 0.9267 - val_loss: 0.3771 - val_accuracy: 0.8433


#### 評估結果

In [13]:
model.load_weights(output_dir+"/weights.02.hdf5")  #請視以上執行結果指定較佳的權重

In [14]:
y_hat = model.predict(x_test)

In [ ]:
plt.hist(y_hat)
_ = plt.axvline(x=0.5, color='orange')

In [ ]:
#註：由於神經網路的初始權重參數是隨機設定的, 參雜了隨機性, 因此底下 (或您重跑一次) 的結果不會與書中完全一樣, 但模型的能力是相近的
"{:0.2f}".format(roc_auc_score(y_test, y_hat)*100.0)